In [13]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install time

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install logging

Defaulting to user installation because normal site-packages is not writeable
  Using cached logging-0.4.9.6.tar.gz (96 kB)
    ERROR: Command errored out with exit status 1:
     command: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/8y/g9jsbl415s54s0bxzztd72p00000gn/T/pip-install-higgsr9z/logging_ca3152c853114a379480ee5421b83299/setup.py'"'"'; __file__='"'"'/private/var/folders/8y/g9jsbl415s54s0bxzztd72p00000gn/T/pip-install-higgsr9z/logging_ca3152c853114a379480ee5421b83299/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/8y/g9jsbl415s54s0bxzztd72p00000gn/T/pip-pip-egg-info-ocl9z6dm
         cwd: /private/var/folders/8y/g9jsbl

In [17]:
pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install pymongo 

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
import logging
import uuid
from pymongo import mongoclient, errors
from selenium import webdriver
from selenium.webdriver.chrome.options import options
from selenium.webdriver.chrome.service import service
from driver_manager.chrome import chromedrivermanager
from bs4 import beautifulsoup

# mongodb configuration
mongo_uri = "mongodb://localhost:27017/"
database_name = "medium_scraper"
collection_name = "repositories"

client = mongoclient(mongo_uri)
db = client[database_name]
collection = db[collection_name]

# set up logging
logger = logging.getlogger(__name__)
logging.basicconfig(level=logging.info)

class mediumcrawler:
    def __init__(self):
        # set up chrome options
        self.options = options()
        self.options.add_argument("--headless")  # run in headless mode

        # use chromedrivermanager with service
        service = service(chromedrivermanager().install())
        self.driver = webdriver.chrome(service=service, options=self.options)

    def scroll_page(self, scroll_pause_time=1):
        """scroll down the page to load more content."""
        last_height = self.driver.execute_script("return document.body.scrollheight")

        while true:
            # scroll down to the bottom
            self.driver.execute_script("window.scrollto(0, document.body.scrollheight);")
            time.sleep(scroll_pause_time)

            # check if we've reached the bottom
            new_height = self.driver.execute_script("return document.body.scrollheight")
            if new_height == last_height:
                break
            last_height = new_height

    def extract(self, link: str, user: dict = none):
        # check if the article already exists in the database
        if collection.find_one({"link": link}):
            logger.info(f"article already exists in the database: {link}")
            return

        logger.info(f"starting to scrape medium article: {link}")

        self.driver.get(link)
        self.scroll_page()

        soup = beautifulsoup(self.driver.page_source, "html.parser")
        title = soup.find_all("h1", class_="pw-post-title")
        subtitle = soup.find_all("h2", class_="pw-subtitle-paragraph")

        data = {
            "title": title[0].get_text(strip=true) if title else none,
            "subtitle": subtitle[0].get_text(strip=true) if subtitle else none,
            "content": soup.get_text(strip=true),
        }

        # save the article data to mongodb
        doc = {
            "_id": str(uuid.uuid4()),
            "link": link,
            "platform": "medium",
            "content": data,
        }

        if user:
            doc["author_id"] = user["id"]
            doc["author_full_name"] = user["full_name"]

        try:
            collection.insert_one(doc)
            logger.info(f"successfully scraped and saved article: {link}")
        except errors.pymongoerror as e:
            logger.error(f"failed to save article to mongodb: {e}")

        return data

    def close(self):
        """close the driver after scraping is done."""
        self.driver.quit()

# example usage:
crawler = mediumcrawler()
sample_user = {"id": str(uuid.uuid4()), "full_name": "sample user"}
article_link = "https://medium.com/@tetraengnrng/a-beginners-guide-to-ros2-29721dcf49c8"
data = crawler.extract(article_link, user=sample_user)
print(data)
crawler.close()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/krishmurjani/.wdm/drivers/chromedriver/mac64/131.0.6778.87/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Starting to scrape Medium article: https://medium.com/schmiedeone/getting-started-with-ros2-part-1-d4c3b7335c71
INFO:__main__:Successfully scraped and saved article: https://medium.com/schmiedeone/getting-started-with-ros2-part-1-d4c3b7335c71


{'Title': None, 'Subtitle': None, 'Content': 'Just a moment...medium.comVerifying you are human. This may take a few seconds.medium.com needs to review the security of your connection before proceeding.Verification successfulWaiting for medium.com to respond...Enable JavaScript and cookies to continueRay ID:8ee329206b3641c3Performance & security byCloudflare'}
